# Deep Learning for text: transformers

In [ ]:
!pip install tensorflow==2.8

In [ ]:
import tensorflow as tf
import keras
print('Tensorflow version: {}'.format(tf.__version__))
print('Keras version: {}'.format(keras.__version__))